In [23]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
from sqlalchemy import create_engine, types
from sqlalchemy import text 

In [ ]:
df_1 = pd.read_csv('../data/Cleaned/cleaned_world_marriage.csv')
df_2 = pd.read_csv('../data/Cleaned/cleaned_age_at_marriage_women.csv')
df_3 = pd.read_csv('../data/Cleaned/cleaned_marriage-rate-per-1000-inhabitants.csv')
df_4 = pd.read_csv('../data/Cleaned/cleaned_marriage-rates-in-1990-vs-2020.csv')
df_5 = pd.read_csv('../data/Cleaned/cleaned_share-of-births-outside-marriage.csv')
df_6 = pd.read_csv('../data/Cleaned/cleaned_share-of-men-in-england-and-wales-who-have-ever-married-by-age.csv')
df_7 = pd.read_csv('../data/Cleaned/cleaned_share-of-single-parent-households.csv')
df_8 = pd.read_csv('../data/Cleaned/cleaned_share-of-women-in-england-and-wales-who-have-ever-married-by-age.csv')
df_9 = pd.read_csv('../data/Cleaned/cleaned_unpopulation_dataportal.csv')
df_10 = pd.read_csv('../data/processed/countries_UN.csv',  header=5, low_memory=False)

In [25]:
df_1.sample(5)

,country,age_group,sex,marital_status,data_process,data_collection_start_year,data_collection_end_year,data_source
257654,United Republic of Tanzania,[35-39],Woman,Married,Census,2012,2012,UNSD
151963,Malawi,[30-34],Man,Married,Census,2018,2018,National statistics
212571,San Marino,[25-29],Man,Widowed,Estimate,2005,2005,UNSD
247265,Trinidad and Tobago,[40-44],Man,Single,Census,1970,1970,US Census Bureau
208479,Saint Vincent and the Grenadines,[60-64],Woman,Widowed,Census,2012,2012,National statistics


In [26]:
print(df_1.dtypes)

country                       object
age_group                     object
sex                           object
marital_status                object
data_process                  object
data_collection_start_year     int64
data_collection_end_year       int64
data_source                   object
dtype: object


In [27]:
df_2.sample(5)

,country,code,year,mean_age_of_women_at_first_marriage
398,Hungary,HUN,2009,28.0
288,Finland,FIN,2005,29.4
434,Ireland,IRL,1993,27.5
981,Turkey,TUR,2017,24.8
593,Luxembourg,LUX,2009,29.7


In [28]:
df_2.dtypes

country                                 object
code                                    object
year                                     int64
mean_age_of_women_at_first_marriage    float64
dtype: object

In [29]:
df_3.sample(5)

,country,code,year,crude_marriage_rate_marriages_per_1000_people
501,Czechia,CZE,1968,9.0
1426,Malta,MLT,1980,8.8
1197,Japan,JPN,1995,6.4
1976,Slovenia,SVN,2004,3.3
1950,Slovenia,SVN,1978,8.4


In [30]:
df_4.sample(6)

,country,code,year,crude_marriage_rate,crude_marriage_rate_people1,year_1
1145,Japan,JPN,2010,5.5,5.9,1990
221,Bulgaria,BGR,2017,4.0,6.9,1990
319,Costa Rica,CRI,2005,6.1,7.5,1990
2014,Spain,ESP,2001,5.1,5.7,1990
5,Australia,AUS,1984,7.0,6.9,1990
2108,Switzerland,CHE,1975,5.6,6.9,1990


In [31]:
df_4.sample(6)

,country,code,year,crude_marriage_rate,crude_marriage_rate_people1,year_1
781,Greece,GRC,1985,6.4,5.8,1990
1566,Norway,NOR,1970,7.6,5.2,1990
1907,Slovenia,SVN,2006,3.2,4.3,1990
20,Australia,AUS,1999,6.0,6.9,1990
160,Belgium,BEL,2017,3.9,6.5,1990
1059,Italy,ITA,1983,5.4,5.6,1990


In [32]:
df_5.sample(5)

,country,code,year,share_of_births_outside_of_marriage
898,Iceland,ISL,1994,59.6
1558,Poland,POL,2008,19.9
1000,Italy,ITA,1961,2.4
2104,United States,USA,1986,23.4
180,Bulgaria,BGR,1974,9.6


In [33]:
df_6.sample(10)

,sex,year,1900_birthcohort,1920_birthcohort,1940_birthcohort,1960_birthcohort,1970_birthcohort
58,Women,41,82.9,90.7,95.1,85.7,72.7
25,Men,42,90.7,90.6,91.6,79.1,65.8
47,Women,30,71.6,82.7,90.9,76.7,55.8
7,Men,24,30.2,34.4,49.8,33.7,14.8
32,Men,49,92.7,91.8,92.3,81.5,70.3
66,Women,49,85.2,91.9,95.7,87.3,76.0
41,Women,24,40.8,56.1,75.5,55.1,29.7
30,Men,47,92.3,91.6,92.2,81.0,69.4
61,Women,44,84.0,91.3,95.4,86.5,74.1
52,Women,35,79.0,88.3,93.8,82.8,66.7


In [34]:
df_7.sample(10)

,country,code,year,share_of_single_parent_households
143,Colombia,COL,2015,12.70
262,Guinea,GIN,2005,5.20
270,Guyana,GUY,2005,9.71
278,Haiti,HTI,2003,10.82
541,Paraguay,PRY,1962,11.06
0,Afghanistan,AFG,2010,0.77
678,Thailand,THA,2010,8.18
555,Peru,PER,2011,11.05
395,Liberia,LBR,2007,8.72
564,Philippines,PHL,2003,6.15


In [35]:
df_8.sample(10)

,sex,year,1900_birthcohort,1920_birthcohort,1940_birthcohort,1960_birthcohort,1970_birthcohort
42,Women,25,48.5,61.8,80.6,60.7,35.2
23,Men,40,89.6,89.9,91.1,78.0,64.0
55,Women,38,81.2,89.8,94.6,84.5,70.5
57,Women,40,82.4,90.5,95.0,85.3,72.0
58,Women,41,82.9,90.7,95.1,85.7,72.7
44,Women,27,60.4,73.4,86.6,68.9,44.7
18,Men,35,84.8,86.7,89.1,73.7,56.3
46,Women,29,68.7,80.4,89.8,74.5,52.4
9,Men,26,48.3,51.0,67.7,46.8,24.1
8,Men,25,39.6,41.6,59.7,40.6,19.4


In [36]:
df_9.sample(5)

,indicatorname,year,country,code,time,variant,sex,age,estimate_method,estimatemethod,value
7069,Currently married (Percent),2024,Mauritius,MUS,1983,Median,Female,15-49,2,Interpolation,58.86
2587,Currently married (Percent),2024,Cook Islands,COK,1981,Median,Female,15-49,2,Interpolation,52.57
4023,Currently married (Percent),2024,Georgia,GEO,2017,Median,Female,15-49,2,Interpolation,65.16
4257,Currently married (Percent),2024,Gibraltar,GIB,1971,Median,Female,15-49,2,Interpolation,68.61
6200,Currently married (Percent),2024,Lesotho,LSO,2010,Median,Female,15-49,2,Interpolation,52.57


In [37]:
df_10.sample(8)

,Country or area,ISO code,Indicator,Year,AgeGroup,Percentage,Number,DataProcess
47200,Gambia,270,Married or in-union women,2038,15-19,10.203811,18.682821,Projection
119991,Somalia,706,Married or in-union women,1983,15-49,67.341111,934.648149,Estimate
133373,Türkiye,792,Married or in-union women,2036,40-44,77.349145,2536.400291,Projection
108548,Russian Federation,643,Married or in-union women,2011,35-39,74.881250,3990.282533,Estimate
97115,Northern Mariana Islands,580,Married or in-union women,2040,30-34,43.466811,0.691774,Projection
86727,Morocco,504,Married or in-union women,2037,15-49,53.291515,5385.508835,Projection
65343,Japan,392,Married or in-union women,2037,15-49,44.034879,8760.257583,Projection
4515,Azerbaijan,31,Married or in-union women,2048,30-34,67.847781,267.139105,Projection
